IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M4\FAULT_M4_10


In [3]:
guasto=2

VOLO 34 FAULT 10%

In [5]:
rcou_m4_fault10 = pd.read_csv("RCOU.csv")
rcou_m4_fault10 = rcou_m4_fault10.astype("float64")
rcou_m4_fault10 = rcou_m4_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m4_fault10 = rcou_m4_fault10[((rcou_m4_fault10['C9'] >= potenza) & (rcou_m4_fault10['C10'] >= potenza) & (rcou_m4_fault10['C11'] >= potenza) & (rcou_m4_fault10['C12'] >= potenza) & (rcou_m4_fault10['C13'] >= potenza) & (rcou_m4_fault10['C14']>= potenza))]
rcou_m4_fault10=rcou_m4_fault10.reset_index(drop=True)
display(rcou_m4_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,71672974.0,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0
1,71717439.0,1535.0,1415.0,1486.0,1471.0,1540.0,1410.0
2,71727490.0,1354.0,1591.0,1516.0,1453.0,1389.0,1567.0
3,71738394.0,1353.0,1604.0,1506.0,1478.0,1368.0,1595.0
4,71751266.0,1407.0,1583.0,1550.0,1448.0,1461.0,1539.0
...,...,...,...,...,...,...,...
5611,140815408.0,1599.0,1404.0,1593.0,1412.0,1559.0,1455.0
5612,140825207.0,1593.0,1406.0,1594.0,1404.0,1558.0,1450.0
5613,140836705.0,1581.0,1415.0,1590.0,1403.0,1543.0,1461.0
5614,140848285.0,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0


In [6]:
min=rcou_m4_fault10['TimeUS'][0]
max=rcou_m4_fault10['TimeUS'][len(rcou_m4_fault10)-1]
print(max)
print(min)

140859141.0
71672974.0


In [7]:
xkf1_m4_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault10 = xkf1_m4_fault10.astype("float64")
xkf1_m4_fault10 = xkf1_m4_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault10 = xkf1_m4_fault10[((xkf1_m4_fault10['TimeUS'] >= min) & (xkf1_m4_fault10['TimeUS'] <= max	))]
xkf1_m4_fault10 = xkf1_m4_fault10.reset_index(drop=True)
print(xkf1_m4_fault10)

           TimeUS  Roll  Pitch     Yaw
0      71683864.0 -0.46  -0.40  355.35
1      71694656.0 -0.10  -0.45  355.36
2      71704946.0  0.10  -0.47  355.35
3      71715377.0  0.05  -0.48  355.34
4      71725387.0 -0.14  -0.40  355.35
...           ...   ...    ...     ...
6282  140813337.0  0.45   4.06    0.42
6283  140823342.0  0.49   3.88    0.36
6284  140834825.0  0.52   3.74    0.39
6285  140846172.0  0.56   3.73    0.45
6286  140856511.0  0.63   3.68    0.55

[6287 rows x 4 columns]


In [8]:
att_m4_fault10 = pd.read_csv("ATT.csv")
att_m4_fault10 = att_m4_fault10.astype("float64")
att_m4_fault10 = att_m4_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault10 = att_m4_fault10[((att_m4_fault10['TimeUS'] >= min) & (att_m4_fault10['TimeUS'] <= max	))]
att_m4_fault10=att_m4_fault10.reset_index(drop=True)
print(att_m4_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      71685046.0     0.07 -0.46      0.18  -0.40  355.26  355.36
1      71695745.0     0.04 -0.10      0.17  -0.45  355.26  355.37
2      71706645.0     0.03  0.10      0.16  -0.47  355.26  355.36
3      71716773.0     0.02  0.05      0.15  -0.48  355.26  355.35
4      71726805.0     0.00 -0.14      0.14  -0.40  355.26  355.36
...           ...      ...   ...       ...    ...     ...     ...
6282  140814739.0     1.13  0.45      3.87   4.06    1.35    0.42
6283  140824626.0     1.13  0.49      3.77   3.88    1.35    0.36
6284  140835923.0     1.12  0.52      3.70   3.74    1.35    0.39
6285  140847431.0     1.11  0.56      3.62   3.73    1.34    0.45
6286  140858295.0     1.09  0.63      3.54   3.68    1.34    0.55

[6287 rows x 7 columns]


In [9]:
from numpy import empty

if len(att_m4_fault10)>len(xkf1_m4_fault10):
    to_add=att_m4_fault10[len(xkf1_m4_fault10):]
    att_m4_fault10=att_m4_fault10[:len(xkf1_m4_fault10)]

if len(xkf1_m4_fault10)>len(att_m4_fault10):
    to_add=xkf1_m4_fault10[len(att_m4_fault10):]

for idx,i in enumerate(att_m4_fault10['Roll']):
    if(xkf1_m4_fault10['Roll'][idx] != i):
        att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Pitch']):
    if(xkf1_m4_fault10['Pitch'][idx] != i):
        att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Yaw']):
    if(xkf1_m4_fault10['Yaw'][idx] != i):
        att_m4_fault10['Yaw'][idx] = ((att_m4_fault10['Yaw'][idx] + xkf1_m4_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault10=pd.concat([att_m4_fault10,to_add])

print(att_m4_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw      Yaw
0      71685046.0     0.07 -0.46      0.18  -0.40  355.26  355.355
1      71695745.0     0.04 -0.10      0.17  -0.45  355.26  355.365
2      71706645.0     0.03  0.10      0.16  -0.47  355.26  355.355
3      71716773.0     0.02  0.05      0.15  -0.48  355.26  355.345
4      71726805.0     0.00 -0.14      0.14  -0.40  355.26  355.355
...           ...      ...   ...       ...    ...     ...      ...
6282  140814739.0     1.13  0.45      3.87   4.06    1.35    0.420
6283  140824626.0     1.13  0.49      3.77   3.88    1.35    0.360
6284  140835923.0     1.12  0.52      3.70   3.74    1.35    0.390
6285  140847431.0     1.11  0.56      3.62   3.73    1.34    0.450
6286  140858295.0     1.09  0.63      3.54   3.68    1.34    0.550

[6287 rows x 7 columns]


In [12]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m4_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault10 = esc_0_m4_fault10.astype("float64")
esc_0_m4_fault10 = esc_0_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault10 = esc_0_m4_fault10[((esc_0_m4_fault10['TimeUS'] >= min) & (esc_0_m4_fault10['TimeUS'] <= max))]
esc_0_m4_fault10=esc_0_m4_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m4_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m4_fault10 = esc_1_m4_fault10.astype("float64")
esc_1_m4_fault10 = esc_1_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m4_fault10 = esc_1_m4_fault10[((esc_1_m4_fault10['TimeUS'] >= min) & (esc_1_m4_fault10['TimeUS'] <= max))]
esc_1_m4_fault10=esc_1_m4_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m4_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m4_fault10 = esc_2_m4_fault10.astype("float64")
esc_2_m4_fault10 = esc_2_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m4_fault10 = esc_2_m4_fault10[((esc_2_m4_fault10['TimeUS'] >= min) & (esc_2_m4_fault10['TimeUS'] <= max))]
esc_2_m4_fault10=esc_2_m4_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m4_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m4_fault10 = esc_3_m4_fault10.astype("float64")
esc_3_m4_fault10 = esc_3_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m4_fault10 = esc_3_m4_fault10[((esc_3_m4_fault10['TimeUS'] >= min) & (esc_3_m4_fault10['TimeUS'] <= max))]
esc_3_m4_fault10=esc_3_m4_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m4_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m4_fault10 = esc_4_m4_fault10.astype("float64")
esc_4_m4_fault10 = esc_4_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m4_fault10 = esc_4_m4_fault10[((esc_4_m4_fault10['TimeUS'] >= min) & (esc_4_m4_fault10['TimeUS'] <= max))]
esc_4_m4_fault10=esc_4_m4_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m4_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m4_fault10 = esc_5_m4_fault10.astype("float64")
esc_5_m4_fault10 = esc_5_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m4_fault10 = esc_5_m4_fault10[((esc_5_m4_fault10['TimeUS'] >= min) & (esc_5_m4_fault10['TimeUS'] <= max))]
esc_5_m4_fault10=esc_5_m4_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

print(esc_5_m4_fault10['TimeUS'])

57      71700974.0
58      71816400.0
59      71926314.0
60      72030221.0
61      72147810.0
          ...     
625    140213655.0
626    140434863.0
627    140537384.0
628    140654428.0
629    140757134.0
Name: TimeUS, Length: 573, dtype: float64


In [13]:
#valori di accelerazione e del giroscopio
imu_0_m4_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault10 = imu_0_m4_fault10.astype("float64")
imu_0_m4_fault10 = imu_0_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault10 = imu_0_m4_fault10[((imu_0_m4_fault10['TimeUS'] >= min) & (imu_0_m4_fault10['TimeUS'] <= max))]

imu_1_m4_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault10 = imu_1_m4_fault10.astype("float64")
imu_1_m4_fault10 = imu_1_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault10 = imu_1_m4_fault10[((imu_1_m4_fault10['TimeUS'] >= min) & (imu_1_m4_fault10['TimeUS'] <= max))]

imu_2_m4_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault10 = imu_2_m4_fault10.astype("float64")
imu_2_m4_fault10 = imu_2_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault10 = imu_2_m4_fault10[((imu_2_m4_fault10['TimeUS'] >= min) & (imu_2_m4_fault10['TimeUS'] <= max))]

imu_m4_fault10 = pd.concat((imu_0_m4_fault10, imu_1_m4_fault10, imu_2_m4_fault10))
imu_m4_fault10=imu_m4_fault10.groupby(imu_m4_fault10.TimeUS, as_index=False).mean()

display(imu_m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,71675210.0,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380
1,71677358.0,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750
2,71679642.0,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288
3,71684827.0,-0.014572,0.018070,0.016195,-0.339925,-0.366561,-9.819399
4,71690467.0,-0.144416,0.112670,-0.008237,-0.512008,0.856650,-10.545677
...,...,...,...,...,...,...,...
23873,140847239.0,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497
23874,140849891.0,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347
23875,140853397.0,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127
23876,140855323.0,0.233055,-0.229394,0.111900,2.790600,0.246165,-13.841943


In [14]:
#sincronizzazione dei tempi tra att e imu_2
m4_fault10 = pd.merge_ordered(imu_m4_fault10,att_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_0_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_1_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_2_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_3_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_4_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_5_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
m4_fault10 = pd.merge_ordered(m4_fault10, rcou_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")

In [15]:
m4_fault10["TimeUS"] = m4_fault10["TimeUS"] - m4_fault10.iloc[0]["TimeUS"]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"].astype(int)
m4_fault10["Guasto"] = guasto

In [16]:
last_cell = m4_fault10.iloc[-1, m4_fault10.columns.get_loc('TimeUS')]
print(last_cell)

69186167


In [17]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [18]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
24212  69173684
24213  69176541
24214  69179398
24215  69182255
24216  69185112

[24217 rows x 1 columns]


In [19]:
m4_fault10_final = pd.merge_ordered(m4_fault10,df)
m4_fault10_final=m4_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m4_fault10_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.562366 -0.017601  0.058196 -0.412550 -0.252978  -9.731380   
1          2236 -0.562366 -0.017601  0.058196 -0.412550 -0.252978  -9.731380   
2          2857 -0.562366 -0.017601  0.058196 -0.412550 -0.252978  -9.731380   
3          4384 -0.381477  0.089880  0.034272 -0.460087  0.749301 -10.227750   
4          5714 -0.381477  0.089880  0.034272 -0.460087  0.749301 -10.227750   
...         ...       ...       ...       ...       ...       ...        ...   
63464  69182349  0.233055 -0.229394  0.111900  2.790600  0.246165 -13.841943   
63465  69185112  0.233055 -0.229394  0.111900  2.790600  0.246165 -13.841943   
63466  69185197  0.073851 -0.066660  0.152899  2.835630  0.035685 -14.132753   
63467  69185321  0.073851 -0.066660  0.152899  2.835630  0.035685 -14.132753   
63468  69186167  0.073851 -0.066660  0.152899  2.835630  0.035685 -14.132753   

       DesRoll  Roll  DesPitch  ...  Cu

In [20]:
m4_fault10_final = m4_fault10_final[m4_fault10_final.TimeUS.isin(to_be)]

In [21]:
m4_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
2,2857,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
4,5714,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
6,8571,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
7,11428,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63455,69173684,0.145880,-0.330572,0.075562,1.462585,0.881795,-12.665910,1.12,0.52,3.70,...,2.21,5907.0,3.11,1581.0,1415.0,1590.0,1403.0,1543.0,1461.0,2.0
63459,69176541,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0
63461,69179398,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0
63463,69182255,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0


In [22]:
from datetime import timedelta
m4_fault10_final=m4_fault10_final.reset_index(drop=True)
m4_fault10_final['TimeUS'] = pd.to_datetime(m4_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
1,00:00:00.002857,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
2,00:00:00.005714,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
3,00:00:00.008571,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
4,00:00:00.011428,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24212,00:01:09.173684,0.145880,-0.330572,0.075562,1.462585,0.881795,-12.665910,1.12,0.52,3.70,...,2.21,5907.0,3.11,1581.0,1415.0,1590.0,1403.0,1543.0,1461.0,2.0
24213,00:01:09.176541,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0
24214,00:01:09.179398,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0
24215,00:01:09.182255,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0


## Analisi nel tempo e in frequenza

In [23]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [25]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_fault10_final)/350)):
        V1=m4_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [27]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_31900\3165731684.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_31900\3165731684.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.035348,0.027541,1.454767,0.169445,68.0,67.0,7.807059,4.044368,-0.018257,...,5782.771484,3.820028,0.089800,0.409842,3.831731,131.0,130.0,12.776340,4.751661,2
1,1.0,-0.035554,0.027477,1.465293,0.169302,83.0,82.0,8.222915,4.483560,-0.018301,...,5755.282650,3.820597,0.089658,0.420118,3.832279,127.0,126.0,13.187094,5.047005,2
2,2.0,-0.035574,0.027399,1.478155,0.169077,65.0,83.0,8.921253,3.871360,-0.018556,...,5716.386896,3.821161,0.089516,0.430414,3.832824,127.0,126.0,13.576807,5.335926,2
3,3.0,-0.035971,0.027377,1.479001,0.169098,78.0,100.0,8.070182,3.916215,-0.018125,...,5666.217336,3.821723,0.089374,0.440729,3.833365,127.0,126.0,13.945349,5.616895,2
4,4.0,-0.036411,0.027369,1.476947,0.169167,79.0,78.0,7.845966,4.003331,-0.017563,...,5604.924405,3.822282,0.089232,0.451063,3.833904,121.0,120.0,14.292633,5.888411,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,64.0,-0.043521,0.024754,1.847637,0.163061,0.0,91.0,8.863475,8.566053,0.000469,...,5032.957048,3.896843,0.110035,0.310906,3.910902,137.0,139.0,6.109480,5.942656,2
65,65.0,-0.043452,0.024697,1.857420,0.162866,96.0,0.0,9.848774,9.033483,0.000556,...,4806.261417,3.897788,0.110141,0.310726,3.911858,133.0,135.0,5.873873,5.829529,2
66,66.0,-0.043383,0.024639,1.867206,0.162673,0.0,87.0,9.202902,8.167168,0.000643,...,4553.657750,3.898729,0.110245,0.310610,3.912808,134.0,133.0,6.105113,5.679271,2
67,67.0,-0.043249,0.024588,1.873802,0.162479,0.0,89.0,9.399085,6.719094,0.000827,...,4591.252343,3.899665,0.110347,0.310556,3.913754,134.0,133.0,6.669822,5.494496,2


In [28]:
path_file = path_file.replace(r"M4\FAULT_M4_10", "")
os.chdir(path_file)
df_merged.to_csv('m4_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
